In [119]:
import pandas as pd
import numpy  as  np
from datetime import datetime, timedelta

In [5]:
cd ..

/Users/ankitrawat/Desktop/smu/Classes/Term5/QF_621_QuantStrategies/Project


In [120]:
fx_df = pd.read_csv("G10_FX_CLOSE.csv")
fx_df["DATE"] = pd.to_datetime(fx_df["DATE"])
fx_df = fx_df.set_index('DATE')
fx_df.columns
fx_df = fx_df.iloc[:,-10:-1]

In [121]:
fx_df.columns = ['JPY','GBP','CAD','AUD', 'NZD', 'CHF', 'DKK', 'NOK','SEK']
fx_df.head(2)

,JPY,GBP,CAD,AUD,NZD,CHF,DKK,NOK,SEK
DATE,,,,,,,,,
2011-03-01,81.74,0.6456,0.9938,0.9835,1.2930,0.9334,5.5788,5.8277,6.7004
2011-04-01,82.04,0.6416,0.9983,0.9949,1.3045,0.9486,5.6006,5.8631,6.7233


In [122]:
ppp_df = pd.read_csv("PPP.csv")
ppp_df['Year']=pd.to_datetime(ppp_df["Year"], format='%Y')
ppp_df = ppp_df.set_index('Year')
ppp_df.index = ppp_df.index.year
ppp_df.head(2)

,GBP,SEK,DKK,JPY,NZD,AUD,CAD,NOK,CHF
Year,,,,,,,,,
1990,0.632935,8.886935,9.144147,187.710514,1.538590,1.365519,1.241965,9.599692,1.936289
1991,0.651530,9.305145,9.080259,186.886354,1.492342,1.348412,1.237998,9.491041,1.974260


In [123]:
fx_df_nominal = fx_df.copy()
for i in fx_df.index.year.unique():
    for j in fx_df.columns:
        fx_df[j][fx_df.index.year == i] = (fx_df[j][fx_df.index.year == i])/(ppp_df[j][ppp_df.index == int(i-1)].values)

In [124]:
fx_df_nominal.head(2) , fx_df.head(2)
train_size = int(fx_df.shape[0] * 0.8)

# Data Split   -  Validation Datat
fx_df_nominal_test = fx_df_nominal.iloc[:train_size,:]
fx_df_nominal_train = fx_df_nominal.iloc[train_size:,:]



# Data Split - Real  - Test Data
fx_df_real_test = fx_df.iloc[:train_size,:]
fx_df_real_train = fx_df.iloc[train_size:,:]


# Generating returns
returns = fx_df_nominal_test/fx_df_nominal_test.shift(-1) - 1
returns.dropna()

signal_temp = fx_df_nominal_test/fx_df_real_test - 1
signal_temp

signal = pd.DataFrame(np.zeros(shape = signal_temp.shape), columns = signal_temp.columns, index = signal_temp.index)
top_n = 2


for index, row in signal_temp.iterrows():
        signal.loc[index, row.nlargest(top_n).index] = 1      
for index, row in signal_temp.iterrows():
        signal.loc[index, row.nsmallest(top_n).index] = -1
        
asset_returns = signal * returns
portfolio_returns = (asset_returns.sum(axis = 1))/4
cum_returns = np.cumprod((1 + portfolio_returns))
cum_returns = cum_returns.shift(1)
cum_returns[0] = 1

In [11]:
from sklearn.preprocessing import StandardScaler
def GetTradingSignal(Close_price):
    '''
    TREND Trading Signal
    - Uses the t-statistics of historical daily log-returns to reflect the strength of price movement trend
    - TREND Signal Conditions:
    t-stat > 1 => TREND Signal = 1
    t-stat < 1 => TREND Signal = -1
    -1 < t-stat < 1 => TREND Signal = t-stat
    '''
    log_returns = np.log(Close_price/Close_price.shift(1)).dropna()
    mean = np.mean(log_returns)
    std = np.std(log_returns)
    n = len(log_returns)
    t_stat = mean/(std/np.sqrt(n))

    # cap holding at 1 and -1
    return np.clip(t_stat, a_max=1, a_min=-1)
GetTradingSignal(fx_df).sort_values()

GBP    0.383823
NZD    0.388121
CHF    0.647124
NOK    0.767882
SEK    0.802781
DKK    0.914946
CAD    0.915690
AUD    0.944837
JPY    1.000000
dtype: float64

In [12]:
data = StandardScaler().fit_transform(GetTradingSignal(fx_df).values.reshape(-1,1))

In [13]:
final_Normalized_weight = []
symbol_port = fx_df_real_test.columns
for i,j in zip(symbol_port,data):
    final_Normalized_weight.append([i,j])
    
final_Normalized_weight

[['JPY', array([1.13098805])],
 ['GBP', array([-1.67553135])],
 ['CAD', array([0.74698059])],
 ['AUD', array([0.87973414])],
 ['NZD', array([-1.65595769])],
 ['CHF', array([-0.4762685])],
 ['DKK', array([0.74359086])],
 ['NOK', array([0.07375369])],
 ['SEK', array([0.23271021])]]

In [ ]:
#Cummulative Return
import matplotlib.pyplot as plt
cum_return = cum_returns[cum_returns.index[-1]]
dates = cum_returns.index
print ("------------------------PnL Graph---------------------------")
plt.plot(dates,(cum_returns-1)*100)
plt.xlabel("Years")
plt.ylabel("Cumulative Returns [%]")
plt.title("Cumulative Return Graph")
plt.grid(axis='y', alpha=0.5)
plt.show()
print('\n')
print("Cumulative Return = ","{:.2%}".format(cum_return-1))

#print mean,std,shape
print ('\n')
print('-----------------Daily Returns Statistics--------------------')
print ('\n')
returns = portfolio_returns.to_frame()
#returns1 = portfolio_returns
my_list = returns1[0].tolist()
x = my_list*100
mean = np.mean(x)
##print(x)
plt.hist(x)
plt.xlabel("Daily Returns in %")
plt.ylabel("Frequency")
#plt.axis([-10, 12, 0, 40])
plt.grid(axis='y', alpha=0.5)
plt.show()
print('Average Daily Returns = ',"{:.2%}".format(mean))
std = returns.std()
print('Standard Deviation of Daily Returns = ', std[0])
skewness = skew(returns)
print('Skewness of Daily Returns = ', skewness)
kurt = kurtosis(returns)
print('Kurtosis of Daily Returns = ', kurt)

print ('\n')
print ('-------------------Annualized Performance---------------------')
print ('\n')

#Annualized Return
ann_returns = (cum_returns[-1] ** (12.00/len(cum_returns)) - 1)
print("Annualized Return = ","{:.2%}".format(ann_returns))

#Sharpe Ratio
drfr = ((1+risk_free_rate)**(1/365))-1
sharpe_ratio = (mean-drfr)/(std)
print("Sharpe Ratio = ",sharpe_ratio[0])

In [ ]:
""" fx_df_nominal.head(2) , fx_df.head(2)
train_size = int(fx_df.shape[0] * 0.8)

# Data Split   -  Validation Datat
fx_df_nominal_test = fx_df_nominal.iloc[:train_size,:].iloc[-1].sort_values(ascending=False)
fx_df_nominal_train = fx_df_nominal.iloc[train_size:,:].iloc[-1].sort_values(ascending=False)



# Data Split - Real  - Test Data
fx_df_real_test = fx_df.iloc[:train_size,:].iloc[-1].sort_values(ascending=False)
fx_df_real_train = fx_df.iloc[train_size:,:].iloc[-1].sort_values(ascending=False)"""

In [14]:
fx_df
expected_returns = [0.4] * 10
len(fx_df.corr())

9

In [13]:
from scipy.optimize import minimize, Bounds, LinearConstraint
#import cvxpy as cvx
W = np.ones((factor_moments.shape[0],1))*(1.0/factor_moments.shape[0])

NameError: name 'factor_moments' is not defined

In [ ]:
W = np.ones((factor_moments.shape[0],1))*(1.0/factor_moments.shape[0])

# Function that runs optimizer
def optimize(func, W, exp_ret, cov, target_return):
    opt_bounds = Bounds(0, 1)
    opt_constraints = ({'type': 'eq',
                        'fun': lambda W: 1.0 - np.sum(W)},
                       {'type': 'eq',
                        'fun': lambda W: target_return - W.T@exp_ret})
    optimal_weights = minimize(func, W, 
                               args=(exp_ret, cov),
                               method='SLSQP',
                               bounds=opt_bounds,
                               constraints=opt_constraints)
    return optimal_weights['x']

In [ ]:
def optimization(self,regressed_df):

        w_big_lo, w_small_lo, w_big_hi, w_small_hi = [], [], [], []
        agg_alpha = []
        status = []
        lambd = 0.1 # cash reserve as a proportion of shorting stocks

        for i in np.arange(0,len(regressed_df),1): # len(df)-1

            # cvx variables of individual weights
            p1 = cvx.Variable() # short big cap lo weight
            p2 = cvx.Variable() # short small cap lo weight
            p3 = cvx.Variable() # long big cap hi weight
            p4 = cvx.Variable() # long small cap hi weight

            # sum of weight * beta = 0, p3 + p4 - 0.1*(p1 + p2) == 1
            constraints = [p1 <= 0, p2 <= 0, p3 >= 0, p4 >= 0,\
                           sum([p3*regressed_df['big_hi_beta'][i], p4*regressed_df['small_hi_beta'][i], \
                                p1*regressed_df['big_lo_beta'][i], p2*regressed_df['small_lo_beta'][i]]) == 0,\
                           p3 + p4 - 0.1*(p1 + p2) == 1] #

            obj = cvx.Maximize(sum([p1*regressed_df['big_lo_alpha'][i], p2*regressed_df['small_lo_alpha'][i], \
                                    p3*regressed_df['big_hi_alpha'][i], p4*regressed_df['small_hi_alpha'][i]]))
            
            prob = cvx.Problem(obj, constraints)
            prob.solve() 
            status.append(prob.status); agg_alpha.append(prob.value); w_big_lo.append(p1.value); \
            w_small_lo.append(p2.value); w_big_hi.append(p3.value); w_small_hi.append(p4.value)

        weights_table = pd.DataFrame({'status': status, 'agg_alpha': agg_alpha, 'w_big_lo': w_big_lo, \
                                      'w_small_lo': w_small_lo, 'w_big_hi': w_big_hi, 'w_small_hi': w_small_hi})
        return weights_table

In [ ]:
fx_df

In [ ]:
import random
df_ = fx_df.pct_change().dropna()
df_mean  = df_.mean()
df_cov  = df_.cov()
num_stocks = len(df_cov)
weights  = np.array([[0.383823,0.388121,0.647124,0.767882, 0.802781,0.914946,0.915690,0.944837,0.954837]])

expected_return = []
port_var = []
for i in range (10):
        weights = np.array([[random.uniform(-1, 1) for j in range(num_stocks)]])
        mu = np.array([0.4]*len(df_cov)) # mu is the vector of expected returns.
        expected_return_port = np.sum(np.reshape(mu,(-1,1)).T * weights)
        expected_return.append(expected_return_port)
        expected_var = np.sqrt(np.dot(weights, np.dot(df_cov, weights.T)))
        port_var.append(expected_var)
        sharpe = expected_return_port/expected_var
        plt.scatter(expected_var[0][0],expected_return_port,c = (expected_return_port /expected_var[0][0]), marker='o')
        
# Convert lists to arrays
port_return = np.array(expected_return)
port_vols = np.array(port_var)

def minimize_sharpe(weights):  
    return -portfolio_stats(weights)['sharpe'] 


def minimize_volatility(weights):  
    # Note that we don't return the negative of volatility here because we 
    # want the absolute value of volatility to shrink, unlike sharpe.
    return portfolio_stats(weights)['volatility']

def minimize_return(weights): 
    return -portfolio_stats(weights)['return']
                                     

bounds = tuple((0,1) for x in range(num_stocks))
initializer = num_stocks * [1./num_stocks,]

print (initializer)
print (bounds)

In [ ]:
constraints = ({'type' : 'eq', 'fun': lambda x: np.sum(x) -1})
bounds = tuple((0,1) for x in range(num_stocks))
#initializer = num_assets * [1./num_assets,]
weighting = np.array([[random.uniform(-1, 1) for j in range(num_stocks)]])
print (initializer)
print (bounds)
import scipy.optimize as optimize
optimal_sharpe=optimize.minimize(minimize_sharpe,
                                 weighting,
                                 method = 'SLSQP',
                                 bounds = bounds,
                                 constraints = constraints)
print(optimal_sharpe)

In [ ]:
port_return 
def portfolio_creation(returns = 0.4):
    
    df_ = fx_df.pct_change().dropna()
    df_mean  = df_.mean()
    df_cov  = df_.cov()
    num_stocks = len(df_cov)
    weights = num_stocks * [1./num_stocks,]
    expected_return = []
    port_var = []
    sharpe_ratio= []
    weights 
    for i in range (10):
        weights = np.array([[random.uniform(-1, 1) for j in range(num_stocks)]])
        mu = np.array([returns]*len(df_cov)) # mu is the vector of expected returns.
        expected_return_port = np.sum(np.reshape(mu,(-1,1)).T * weights)
        expected_return.append(expected_return_port)
        expected_var = np.sqrt(np.dot(weights, np.dot(df_cov, weights.T)))
        port_var.append(expected_var)
        sharpe = expected_return_port/expected_var
        sharpe_ratio.append(sharpe)
        plt.scatter(expected_var[0][0],expected_return_port,c = (expected_return_port /expected_var[0][0]), marker='o')

    # Convert lists to arrays
    port_return = np.array(expected_return)
    port_vols = np.array(port_var)
    sharpe_ratio = np.array(sharpe_ratio)
    print(port_return)
    return {'return': port_return, 'volatility': port_vols, 'sharpe': sharpe_ratio}


In [ ]:
import random
df_ = fx_df.pct_change().dropna()
df_mean  = df_.mean()
df_cov  = df_.cov()
num_stocks = len(df_cov)
assets = fx_df.columns
def portfolio_statsss(w):

    # Convert to array in case list was passed instead.
    weights = np.array(w)
    mu = np.array([0.4]*len(df_cov)) # mu is the vector of expected returns.
    expected_return_port = np.sum(np.reshape(mu,(-1,1)).T * weights)
    expected_var = np.sqrt(np.dot(weights, np.dot(df_cov , weights.T)))
    sharpe = expected_return_port/expected_var
    print(port_return)
    return {'return': expected_return_port, 'volatility': expected_var, 'sharpe': sharpe}

def minimize_volatility(weights):  
    # Note that we don't return the negative of volatility here because we 
    # want the absolute value of volatility to shrink, unlike sharpe.
    return portfolio_statsss(weights)['volatility']

def minimize_sharpe(weights):  
    return -portfolio_statsss(weights)['sharpe'] 

def minimize_return(weights): 
    return -portfolio_statsss(weights)['return']
                                     
constraint1 = ({'type' : 'eq', 'fun': lambda x: np.sum(x) -1})
bounds = tuple((-1,1) for x in range(9))
initializer = 9 * [1./9,]
weighting = np.array([[random.uniform(-1, 1) for j in range(num_stocks)]])

port_return = np.array(expected_return)
port_vols = np.array(port_var)

import scipy.optimize as optimize
optimal_sharpe=optimize.minimize(minimize_return,
                                 weighting,
                                 method = 'SLSQP',
                                 bounds = bounds,
                                 constraints = constraints)

optimal_sharpe_weights=optimal_sharpe['x'].round(4)
list(zip(assets,list(optimal_sharpe_weights)))

In [ ]:
# Make an array of 50 returns betweeb the minimum return and maximum return
# discovered earlier.
weighting = np.array([[random.uniform(-1, 1) for j in range(num_stocks)]])
return_exp = [-0.28922705, -0.75743592,  0.79251574,  0.687579  , -0.01509249,-0.05254173,  0.77063604,  0.31677695, -0.45321054]

target_returns = np.linspace(np.min(return_exp),np.max(return_exp),50)

print("taget")
print(target_returns)
print("hello")
# Initialize optimization parameters
minimal_volatilities = []
bounds = tuple((-1,1) for x in range(9))
#initializer = num_assets * [1./num_assets,]

weighting = np.array([[random.uniform(-1, 1) for j in range(num_stocks)]])


for target_return in target_returns:
    
    constraints = ({'type':'eq','fun': lambda x: portfolio_statsss(x)['return']-target_return},
                   {'type':'eq','fun': lambda x: np.sum(x)-1})
       
    optimal = optimize.minimize(minimize_volatility,
                               weighting,
                              method = 'SLSQP',
                              bounds = bounds,
                              constraints = constraints)
    
    minimal_volatilities.append(optimal['fun'])

minimal_volatilities = np.array(minimal_volatilities)
print(minimal_volatilities)
minimal_volatilities = np.array(minimal_volatilities)

In [ ]:
fx_df.iloc[:,:4]

In [ ]:
df_ = fx_df.iloc[:,:4].pct_change().dropna()
df_cov = df_cov.cov()

In [ ]:
import random
df_ = fx_df.iloc[:,:4].pct_change().dropna()
print(df_)
df_mean  = df_.mean()
df_cov  = df_.cov()
num_stocks = len(df_cov)
print(num_stocks)
assets = fx_df.columns



def portfolio_statsss(weights):
    expected_return = 0.4 
    cov_mat = df_.cov()
    # Convert to array in case list was passed instead.
    weights = np.array(weights)
    mu = np.array([0.4]*len(cov_mat))
    expected_return_port = np.sum(np.reshape(mu,(-1,1)).T * weights)
    expected_var = np.sqrt(np.dot(weights, np.dot(df_cov , weights.T)))
    sharpe = expected_return_port/expected_var
    return {'return': expected_return_port, 'volatility': expected_var, 'sharpe': sharpe}

def minimize_volatility(weights):  
    # Note that we don't return the negative of volatility here because we 
    # want the absolute value of volatility to shrink, unlike sharpe.
    return portfolio_statsss(weights)['volatility']

def minimize_sharpe(weights):  
    return -portfolio_statsss(weights)['sharpe'] 

def minimize_return(weights): 
    return -portfolio_statsss(weights)['return']


bounds = tuple((-1,1) for x in range(num_stocks))


weighting = np.array([[random.uniform(-6, 6) for j in range(num_stocks)]])

port_return = np.array(expected_return)
port_vols = np.array(port_var)

"""
def constraint1(x):
    return (x[0]== 0)

def constraint2(x):
    return (x[1]== 0)

def constraint3(x):
    return (x[2]==0)

def constraint4(x):
    return (x[3]==0)
"""

con10 ={'type': 'eq', 'fun': lambda x: np.sum(x) - 1}                                     
con1 ={'type' : 'ineq', 'fun':lambda x: x[0]-0} 
con2 ={'type' : 'ineq', 'fun':lambda x: x[1]-0} 
con3 ={'type' : 'ineq', 'fun':lambda x: 0-x[2]} 
con4 ={'type' : 'ineq', 'fun':lambda x: 0-x[3]} 

con = [con1,con2,con3,con4,con10]

import scipy.optimize as optimize
optimal_sharpe=optimize.minimize(minimize_sharpe,
                                 weighting,
                                 method = 'SLSQP',
                                 bounds = bounds,
                                 constraints = con)

optimal_sharpe_weights=optimal_sharpe['x']
list(zip(assets,list(optimal_sharpe_weights)))

In [ ]:
df_ = fx_df.iloc[:,:4].pct_change().dropna()
cov_mat = df_.cov()
Sigma = cov_mat.values


In [58]:
from cvxpy import *
import cvxpy as cvx
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from yahoo_finance import Share
import random



exp_return = 0.4 
df_ = fx_df.iloc[:,:4].pct_change().dropna()
cov_mat = df_.cov()
Sigma = cov_mat.values # np.asarray(cov_mat.values)


# sum of weight * beta = 0, p3 + p4 - 0.1*(p1 + p2) == 1


w = np.array([float(p1.value),float(p2.value),float(p3.value),float(p4.value)])
mu = np.array([exp_return]*len(cov_mat))

expected_return = np.dot(np.reshape(mu,(-1,1)).T , w) 
ret = mu.T@w 

print("Mu shape - reshape {}".format(np.reshape(mu,(-1,1)).T.shape))
print("Mu shape - mu.T.w {}".format((expected_return.shape)))

risk = w.T*np.dot(Sigma,w)

num_stocks = len(cov_mat)
gamma = Parameter(nonneg=True)
gamma.value = 0.2
risk10 = quad_form(w, Sigma)
objective = cvx.Maximize(expected_return - gamma*risk)

'''
List of Constraints :
1. Total weight = 1
2. Long and Short Portfolio Creation 
3. P1 and P2 short position with weight varying between -1 to 0 
4. P3 and P4 Long position with weight varying between 0 to 1 
'''

constraints = [p1 >= -1,p1 <= 0, p2 <= 0,p2 >= -1,p3 <= 1,p3 >= 0,0 <= p4,p4 <= 1,p3 + p4 - 0.1*(p1 + p2) == 1] #
prob = cvx.Problem(objective, constraints)
prob.solve()

"""
SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)

gamma_vals = np.logspace(-2, 3, num=SAMPLES)

for i in range(SAMPLES):
    print(i)
    gamma.value = gamma_vals[i]
    #print ("Gamme Value : {}".format(gamma.value))
    print("gamma value : {}".format(gamma.value))
    prob.solve()
    print("P1 : {}".format(p1.value))
    risk_data[i] = cp.sqrt(risk10).value
    print(risk_data[i])
    ret_data[i] = expected_return[0]

print(risk_data)
print(ret_data)


# Plot long only trade-off curve.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

markers_on = [29, 40]
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(risk_data, ret_data, 'g-')

for marker in markers_on:
    plt.plot(risk_data[marker], ret_data[marker], 'bs')
    #ax.annotate(r"$\gamma = %.2f$" % gamma_vals[marker], xy=(risk_data[marker]+.08, ret_data[marker]-.03))

# Compute trade-off curve - Efficient Frontier 
SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)
for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    risk_data[i] = cp.sqrt(risk).value
    ret_data[i] = ret.value
"""

Mu shape - reshape (1, 4)
Mu shape - mu.T.w (1,)


ValueError: The 'maximize' objective must resolve to a scalar.

In [18]:
df_.head(10)

,JPY,GBP,CAD,AUD
DATE,,,,
2011-04-01,0.003670,-0.006196,0.004528,0.011591
2011-05-01,0.014749,0.004520,-0.002705,0.005629
2011-06-01,0.000961,0.002793,0.001004,0.005097
2011-07-01,-0.002160,-0.004797,-0.003111,-0.001492
2011-10-01,-0.005292,-0.001710,-0.000101,0.000299
2011-11-01,0.006408,-0.001557,-0.002919,0.008562
2011-12-01,-0.002883,-0.010607,-0.004846,-0.009477
2011-01-13,-0.002289,-0.004572,0.003551,-0.000997
2011-01-14,0.000725,-0.002059,0.001719,0.008879


In [46]:
from cvxpy import *
import cvxpy as cp
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from yahoo_finance import Share
import random

exp_return = 0.4 

df_ = fx_df.iloc[:,:6].pct_change().dropna()
print(df_)
cov_mat = df_.cov()
Sigma = cov_mat.values # np.asarray(cov_mat.values)

w = cp.Variable(len(df_.columns))

mu = np.array([exp_return]*len(cov_mat)) # mu is the vector of expected returns.
mu = mu.reshape(-1,1)

expected_return = mu.T@w 

num_stocks = len(cov_mat)
gamma = Parameter(nonneg=True)
gamma.value = 1

risk = quad_form(w, Sigma)
#, w[1] <= 0,w[2] >= -1,w[3] <= 1,w[3]>= 0,0 <= w[1],w[1] <= 1,
opt_constraints = [w[0] >= -1,w[0] <= 0,w[1] >= -1,w[1] <= 0,w[2] <= 1,w[2] <= 0,w[2] <= 1, \
                   w[3] >= 0,w[3] <= 1,w[4] >= 0,w[4] <= 1,w[5] >= 0,w[5] <= 1, \
                   sum(w) == 1] 

prob = cp.Problem(cp.Maximize(expected_return - gamma*risk),constraints=opt_constraints)
prob.solve()
print("First Stock Weights : {}".format(w[0].value))
print("Second Stock Weights : {} ".format(w[1].value))
print("Third Stock Weights : {}".format(w[2].value))
print("Fourth Stock Weights : {} ".format(w[3].value))
print("Fourth Stock Weights : {} ".format(w[4].value))
print("Fourth Stock Weights : {} ".format(w[5].value))
print ("Total Weight : {}".format( w[0].value + w[1].value + w[2].value + w[3].value + w[4].value + w[5].value))

                 JPY       GBP       CAD       AUD       NZD       CHF
DATE                                                                  
2011-04-01  0.003670 -0.006196  0.004528  0.011591  0.008894  0.016285
2011-05-01  0.014749  0.004520 -0.002705  0.005629  0.012495  0.018237
2011-06-01  0.000961  0.002793  0.001004  0.005097  0.000681 -0.000414
2011-07-01 -0.002160 -0.004797 -0.003111 -0.001492 -0.004237  0.001864
2011-10-01 -0.005292 -0.001710 -0.000101  0.000299 -0.004939  0.000310
...              ...       ...       ...       ...       ...       ...
2021-06-16  0.005723  0.006760  0.007633  0.010147  0.009898  0.011915
2021-06-17 -0.004516  0.004756  0.006598  0.007686  0.006628  0.009684
2021-06-18  0.000000  0.008075  0.008658  0.009742  0.009947  0.004469
2021-06-21  0.000544 -0.008839 -0.008183 -0.007479 -0.007491 -0.004123
2021-06-22  0.001995  0.002229  0.001294  0.002788  0.002166  0.002942

[2731 rows x 6 columns]
First Stock Weights : 1.0254266266328992e-18
Second 

In [1]:
## this is the latest code  - copy

from cvxpy import *
import cvxpy as cvx
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from yahoo_finance import Share
import random

exp_return = 0.4 

df_ = fx_df.iloc[:,:4].pct_change().dropna()
cov_mat = df_.cov()
Sigma = cov_mat.values # np.asarray(cov_mat.values)
w = cp.Variable(4)
p1 = cp.Variable()
p2 = cp.Variable()
p3 = cp.Variable()
p4 = cp.Variable()
p1.value = random.uniform(0, 1) 
p2.value = random.uniform(0, 1) 
p3.value = random.uniform(-1, 0) 
p4.value = random.uniform(-1, 0) 

print("testing variable")
weight_testing = np.array([ p1.value, p2.value,p3.value,p4.value])
weight_testing = weight_testing.reshape(-1,1).T[0]


## Variable 
'''
p1 = cvx.Variable() # short big cap lo weight
p2 = cvx.Variable() # short small cap lo weight
p3 = cvx.Variable() # long big cap hi weight
p4 = cvx.Variable() # long small cap hi weight


p1.value = np.array(random.uniform(-1, 1))
p2.value = np.array(random.uniform(-1, 1))
p3.value = np.array(random.uniform(-1, 1))
p4.value = np.array(random.uniform(-1, 1))
'''

# sum of weight * beta = 0, p3 + p4 - 0.1*(p1 + p2) == 1


#w = np.array([float(p1.value),float(p2.value),float(p3.value),float(p4.value)])
mu = np.array([exp_return]*len(cov_mat)) # mu is the vector of expected returns.
mu = mu.reshape(-1,1)
print("mu wuth reshae {}".format(mu.shape))
print("mu with reshae {}".format(mu))
#expected_return = np.dot(np.reshape(mu,(-1,1)).T , w) 
print(w)
expected_return = mu.T@w 
print("Testing00> for ------------")
print("expected_return")
print("mu without reshae {}".format(mu.shape))
print("mu.T.shape {}",format(mu.T.shape))
print("expected_return : {} ".format(expected_return))
print("expected_return shaoe : {}".format(expected_return.shape))
print(mu.T.shape)



"""Testing """
print("Testing-------")
print("mu.shape {}".format(mu.shape))
print("mu.T.shape {}",format(mu.T.shape))
print("weight_testing : {} ".format(weight_testing.reshape(-1,1).T))
print("weight_testing : {} ".format(weight_testing.reshape(-1,1).T[0].shape))
expected_return = np.dot(mu.T,weight_testing)
print("expected_return")
print(expected_return)
print("expected_return_original : {}".format(expected_return))
print(expected_return.reshape(1,-1))
print("end")

""""""


print(w.shape)  ## this is matching 
print("Mu shape - reshape {}".format(np.reshape(mu,(-1,1)).T.shape))
print("Mu shape - mu.T.w {}".format((expected_return[0].shape)))



#w = Variable(len(cov_mat))  # #number of stocks for portfolio weights
#expected_variance => w.T*C*w =  quad_form(w, C)

#print("Risk shape : {}".format(risk.shape))
print(" ris {}".format(risk.shape))
num_stocks = len(cov_mat)
gamma = Parameter(nonneg=True)
gamma.value = 1
risk = quad_form(w, Sigma)
print("Risk Quand Form")
print("w.shape {}".format(w.shape))
print("Sigma shape {}".format(Sigma.shape))
print("risk shape  {}".format(risk.shape))
print(risk.shape)
print("Gama:")
print(gamma*risk)
print(expected_return)

#constraints = [sum(w)==1] #
#prob = cvx.Problem(objective, constraints)

'''
Long_1 =  w[0].value 
Long_2 =  w[1].value
Short_3 = w[2].value
Short_4 = w[3].value

'''
opt_constraints = [w[0] >= -1,w[0] <= 0, w[1] <= 0,w[2] >= -1,w[3] <= 1,w[3]>= 0,0 <= w[1],w[1] <= 1,w[0] +w[1] - 0.1*(w[2] + w[3]) == 1] #


prob = cp.Problem(cp.Maximize(expected_return - gamma*risk), constraints = opt_constraints)
prob.solve()
print(w[0].value)
print(w[1].value)
print(w[2].value)
print(w[3].value)
print (w[0].value + w[1].value + w[2].value + w[3].value)

NameError: name 'fx_df' is not defined

In [ ]:
#constraints = [Long_1 >= -1,Long_1 <= 0, Short_3 <= 0,Short_3 >= -1,Short_4 <= 1,Short_4 >= 0,0 <= Long_2,Long_2 <= 1,Long_1 + Long_2 - 0.1*(Long_3 + Long_4) == 1] #



In [ ]:
ret_data 

In [ ]:
# Plot long only trade-off curve.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

markers_on = [29, 40]
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(risk_data, ret_data, 'g-')
for marker in markers_on:
    plt.plot(risk_data[marker], ret_data[marker], 'bs')
    ax.annotate(r"$\gamma = %.2f$" % gamma_vals[marker], xy=(risk_data[marker]+.08, ret_data[marker]-.03))

for i in range(4):
    plt.plot(cp.sqrt(Sigma[i,i]).value, mu[i], 'ro')

plt.xlabel('Standard deviation')
plt.ylabel('Return')
plt.show()

In [57]:
import numpy as np
import scipy.sparse as sp
from cvxpy import *
import cvxpy as cp
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from yahoo_finance import Share
np.random.seed(1)
n = 4
#mu = np.abs(np.random.randn(n, 1))
exp_return = 0.4 

df_ = fx_df.iloc[:,:4].pct_change().dropna()
cov_mat = df_.cov()
Sigma = cov_mat.values # np.asarray(cov_mat.values)
w = cp.Variable(4)
mu = np.array([exp_return]*len(cov_mat)) # mu is the vector of expected returns.
mu = mu.reshape(-1,1)
print("mu :{}".format(mu))
Sigma = np.random.randn(n, n)
Sigma = Sigma.T.dot(Sigma)
print("Sigma shape {}".format(Sigma.shape))


w = cp.Variable(n)
gamma = cp.Parameter(nonneg=True)
ret = mu.T@w 
print("w shape")
print(w.shape)
print("ret")
print(ret)
print("Shape mu :{}".format(mu.shape))
print("Shape mu.T :{}".format(mu.T.shape))
print("return : mu.T@w  {}".format(ret.shape))
print(ret.shape)
risk = cp.quad_form(w, Sigma)
print("ris")
print("w shape")
print(w.shape)
print("Shape Sigma :{}".format(Sigma.shape))
print(" ris {}".format(risk.shape))
print(risk.value)
print("ffdgfdjgfdsgfsjhgshg")
print("minus")
gamma.value = 1
print(ret - gamma*risk)
# Compute trade-off curve.
0 <= w[0] <= 1
prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
               [0 <= w[0],w[0] <= 1 ,0 <= w[1],w[1]>= 1,0 >= w[2],w[2] >= 1,0 >= w[3],w[3] >= 1,sum(w) == 1])
prob.solve()
print(w[0].value)
print(w[1].value)
print(w[2].value)
print(w[3].value)

"""  
#Compute trade-off curve.
SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)

for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    risk_data[i] = cp.sqrt(risk).value
    ret_data[i] = ret.value
ret_data
"""

mu :[[0.4]
 [0.4]
 [0.4]
 [0.4]]
Sigma shape (4, 4)
w shape
(4,)
ret
[[0.4 0.4 0.4 0.4]] @ var1942
Shape mu :(4, 1)
Shape mu.T :(1, 4)
return : mu.T@w  (1,)
(1,)
ris
w shape
(4,)
Shape Sigma :(4, 4)
 ris (1, 1)
None
ffdgfdjgfdsgfsjhgshg
minus
[[0.4 0.4 0.4 0.4]] @ var1942 + -param1943 @ QuadForm(var1942, [[ 3.59316702 -2.94120601  0.7529629  -2.70426743]
 [-2.94120601  5.88100961 -4.49267483  3.34449867]
 [ 0.7529629  -4.49267483  6.74652626 -5.02062223]
 [-2.70426743  3.34449867 -5.02062223  7.18463815]])


Exception: Cannot evaluate the truth value of a constraint or chain constraints, e.g., 1 >= x >= 0.

In [15]:
# Plot long only trade-off curve.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

markers_on = [29, 40]
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(risk_data, ret_data, 'g-')
for marker in markers_on:
    plt.plot(risk_data[marker], ret_data[marker], 'bs')
    ax.annotate(r"$\gamma = %.2f$" % gamma_vals[marker], xy=(risk_data[marker]+.08, ret_data[marker]-.03))
for i in range(n):
    plt.plot(cp.sqrt(Sigma[i,i]).value, mu[i], 'ro')
plt.xlabel('Standard deviation')
plt.ylabel('Return')
plt.show()

mu :[[0.4]
 [0.4]
 [0.4]
 [0.4]]
Sigma shape (4, 4)
w shape
(4,)
ret
[[0.4 0.4 0.4 0.4]] @ var5
Shape mu :(4, 1)
Shape mu.T :(1, 4)
return : mu.T@w  (1,)
(1,)
ris
w shape
(4,)
Shape Sigma :(4, 4)
 ris (1, 1)
None
ffdgfdjgfdsgfsjhgshg
minus
[[0.4 0.4 0.4 0.4]] @ var5 + -param6 @ QuadForm(var5, [[ 3.59316702 -2.94120601  0.7529629  -2.70426743]
 [-2.94120601  5.88100961 -4.49267483  3.34449867]
 [ 0.7529629  -4.49267483  6.74652626 -5.02062223]
 [-2.70426743  3.34449867 -5.02062223  7.18463815]])
0.29005095771035366
0.25065707662162917
0.2739586204955916
0.18533334517242542
